In [2]:
import numpy as np
import scipy as sp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import wooldridge
import math
from typing import *
from statsmodels.tools.eval_measures import rmse
sns.set(rc={'figure.figsize':(11.7,8.27)})
df = pd.read_csv("BT2101_week8_hw.csv")
df

,faminc,bwght,fatheduc,motheduc,male,white,cigs
0,13.5,109,12.0,12.0,1,1,0
1,7.5,133,6.0,12.0,1,0,0
2,0.5,129,NaN,12.0,0,0,0
3,15.5,126,12.0,12.0,1,0,0
4,27.5,134,14.0,12.0,1,1,0
...,...,...,...,...,...,...,...
1383,27.5,110,12.0,12.0,1,1,0
1384,5.5,146,NaN,16.0,1,1,0
1385,65.0,135,18.0,16.0,0,1,0
1386,27.5,118,NaN,14.0,0,1,0


# Qns 1

Get some summary statistics about the two variables “bwght” and “cigs” including their means 
and standard deviations. Does the variable “bwght” look like a normal random variable? Hint: a 
normal random variable is symmetric (skewness=0), and its kurtosis is equal to 3. 

In [2]:
print(f"bwght skewness level: {df['bwght'].skew()}")
print(f"bwght kurtosis level: {df['bwght'].kurt()}")

bwght skewness level: -0.14602351629957394
bwght kurtosis level: 3.1633425253063665


# Qns 2

 Get the correlation between the variables “bwght” and “cigs”. Is there a negative relationship 
between the two variables?

In [3]:
y = df['bwght']
x = df['cigs']
model = smf.ols("bwght ~ cigs", data=df).fit()
print(model.summary())
print(model.params[1])
# sns.regplot(x=x, y=y, data=df)

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     32.24
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           1.66e-08
Time:                        18:27:24   Log-Likelihood:                -6135.5
No. Observations:                1388   AIC:                         1.227e+04
Df Residuals:                    1386   BIC:                         1.229e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    119.7719      0.572    209.267      0.0

# Qns 3

Divide the data into two groups. In the first group, mothers don’t smoke during pregnancy, and 
in the second group, mothers smoke during pregnancy. Conduct a difference-in-mean test on these 
two  groups.  Investigate  whether  the  birth  weight  of  infants  from  the  first  group  is  statistically 
different from that of the second group.

In [4]:
mean_cig: float = df[(df['cigs'] > 0)]['bwght'].mean()
mean_no_cig: float = df[(df['cigs'] == 0)]['bwght'].mean()
print(f"mothers with cigs mean birth rate: {mean_cig}")
print(f"mothers without cigs mean birth rate: {mean_no_cig}")
# print the difference
print(f"difference between mean birth rate with and without cigs: {mean_no_cig - mean_cig}")
# t-test
no_cigs = df[(df['cigs'] == 0)]
no_cigs_size = len(no_cigs)

cigs = df[(df['cigs'] > 0)]

cigs_size = len(cigs)
t = (mean_no_cig - mean_cig) / math.sqrt(((df[(df['cigs'] == 0)]['bwght'].std())**2 / len(df[(df['cigs'] == 0)]) + ((df[(df['cigs'] > 0)]['bwght'].std())**2 / cigs_size)))
df = min(cigs_size, no_cigs_size) - 1
pvalue = (1 - sp.stats.t.cdf(t, df)) * 2
print(f"p-value: {pvalue}")

mothers with cigs mean birth rate: 111.14622641509433
mothers without cigs mean birth rate: 120.06122448979592
difference between mean birth rate with and without cigs: 8.914998074701586
p-value: 3.362457867694957e-09


# Qns 4

In this simple regression, what is the estimated effect of smoking on infant health? Is this effect 
large or small? Is it statistically significant the 5% level?  

In [3]:
model = smf.ols("bwght ~ cigs", data=df).fit()
model.summary()
print(f"p-values: {model.pvalues[1]}")

p-values: 1.661538170263396e-08


# Qns 5

You  further  investigate  how much variation in “bwght” can be explained by “cigs”.  The  R-
squared  turns  out  to  be  very  small.  Comment  on  this  statement:  since  the  R-squared  from  this 
simple regression is very small, smoking during pregnancy seems to be irrelevant to infant health; 
in other words, health officials (and others) shouldn’t devote any attention to this issue at all.

In [4]:
print(f"r-squared amount: {model.rsquared}")

r-squared amount: 0.022729121106052963


# Qns 6

You suspect that the simple regression suffers an omitted variable bias problem. Based on the 
data, what are potential omitted variables? Why do they qualify as omitted variables?

# Qns 7

Run  a  multiple  regression  of  “bwght” on “cigs”, “faminc”, “fatheduc”,  “motheduc”, “male”, 
and  “white”.  What  is  the  new  estimated  effect  of  smoking  on  infant  health?  What  do  the 
coefficients of “male” and “white” tell you?  

In [5]:
model = smf.ols(formula=f"bwght ~ cigs + faminc + fatheduc + motheduc + male + white", data=df).fit()
print(model.summary())
print(f"new cigarette coefficient: {model.params[1]}")
print(f"new male coefficient: {model.params[-2]}")
print(f"new white coefficient: {model.params[-1]}")

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     9.798
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           1.47e-10
Time:                        18:27:57   Log-Likelihood:                -5236.9
No. Observations:                1191   AIC:                         1.049e+04
Df Residuals:                    1184   BIC:                         1.052e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    113.0051      3.706     30.490      0.0

# Qns 8

Since “faminc”, “fatheduc”, “motheduc” are not individually significant  at the 5% level, you 
decide  to  conduct  a  test  with  the  null  that  these  three  variables  jointly  have  no  effect  on  infant 
health. What kind of test should you use? Can you reject the null at the 5% significant level? 

In [6]:
model = smf.ols(formula="bwght ~ faminc + fatheduc + motheduc", data=df).fit()
print(model.summary())
print(f"probability of the f statistic {model.f_pvalue}")

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     3.846
Date:                Tue, 15 Mar 2022   Prob (F-statistic):            0.00937
Time:                        18:28:00   Log-Likelihood:                -5260.0
No. Observations:                1191   AIC:                         1.053e+04
Df Residuals:                    1187   BIC:                         1.055e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    113.2260      3.419     33.119      0.0

# Qns 9

Although “faminc” is not significant, you still decide to include it in the regression to avoid the 
omitted variable bias. Higher income families tend to smoke higher quality cigarettes, which tend 
to have a less severe impact on the health of an infant, your further conjecture that the effects of 
smoking  on  infant  health  should  be  decreasing  with  the  family  income.  Run  a  regression  to 
incorporate this new conjecture, including all variables in question 8. Do you see such a decreasing 
effect in the regression? Is it statistically significant at the 5% level?

In [7]:
model = smf.ols(formula="bwght ~ cigs + faminc + fatheduc + motheduc + (faminc * cigs)", data=df).fit()
print(model.summary())
print(f"coeff of the (famic * cigs): {model.params[-1]}")
print(f"p-value for the (faminc * cigs): {model.pvalues[-1]}")

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     8.262
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           1.10e-07
Time:                        18:28:02   Log-Likelihood:                -5245.3
No. Observations:                1191   AIC:                         1.050e+04
Df Residuals:                    1185   BIC:                         1.053e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     118.2532      3.504     33.745      

# Qns 10

You suspect that the effect of  smoking on birth weight is stronger for infant boys (male=1) 
than for infant girls (male=0). Run a  multiple regression to incorporate  this effect,  including  all 
variables  in  question  8.  Does  the  regression  result  suggest  such  an  asymmetric  effect?  Is  it 
statistically significant at the 5% level?

In [8]:
model = smf.ols("bwght ~ cigs + faminc + fatheduc + motheduc + (male * cigs)", data = df).fit()
print(model.summary())
print(f"p-value of (male * cigs): {model.pvalues[-1]}")
print(model.params[1] + model.params[-1])
print(model.params[1])

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     8.432
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           5.59e-09
Time:                        18:28:04   Log-Likelihood:                -5240.8
No. Observations:                1191   AIC:                         1.050e+04
Df Residuals:                    1184   BIC:                         1.053e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    116.2031      3.538     32.841      0.0